In [1]:
import os
import datetime

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

import sys
sys.path.append("/home/dongwonshin/Desktop/Open3D/build/lib")
from py3d import *

from VolumetricVAE import *

# Dataloader

In [2]:
nyudv2_color_codes = {
    'empty': [0,0,0],
    'floor': [162, 227, 148], 
    'wall': [183, 205, 235], 
    'cabinet': [35, 131, 188], 
    'bed': [255, 196, 131], 
    'chair': [195, 197, 37], 
    'sofa': [151, 97, 84], 
    'table': [255, 162, 161], 
    'door': [218, 45, 45], 
    'window': [203, 185, 218], 
    'bookshelf': [159, 114, 196], 
    'picture': [203, 166, 158], 
    'counter': [24, 197, 214], 
    'desk': [248, 191, 216], 
    'curtain': [222, 224, 152], 
    'refrigerator': [254, 139, 10], 
    'bathtub':[231, 130, 201], 
    'shower_curtain': [168, 223, 232], 
    'toilet': [48, 170, 51], 
    'sink':[123, 139, 155], 
    'otherfuniture':[93, 93, 173] 
}
    
def rgb2nyudv2(rgb):
    
    color_dist = []
    for class_name, code in nyudv2_color_codes.items():
        color_dist.append(np.linalg.norm(np.asarray(code)-rgb))
    
    class_id = np.argmin(color_dist)
    class_name = list(nyudv2_color_codes.keys())[class_id]
    
    return class_id, class_name

In [3]:
def pcd2voxel(pcd, filter_rad=0.1, dim_size=32):
    [xc, yc, zc] = np.mean(pcd.points,0)
    [xs, ys, zs] = [xc, yc, zc] + np.ones(3)*filter_rad*dim_size/2
    [xe, ye, ze] = [xc, yc, zc] - np.ones(3)*filter_rad*dim_size/2

    voxel_volume = np.zeros((dim_size,dim_size,dim_size))
    color_volume = np.zeros((dim_size,dim_size,dim_size,3))
    for point, color in zip(pcd.points, pcd.colors):
        xi,yi,zi = (np.around(([xs, ys, zs] - point)/filter_rad)).astype(int)
        voxel_volume[xi, yi, zi] = 1
        color_volume[xi, yi, zi]= color
    
    return voxel_volume, color_volume

def colorvol2classidvol(color_volume, onehot=True):
    classid_volume = np.zeros([32,32,32])
    classid_volume = classid_volume.flatten()
    color_volume = color_volume.reshape([-1,3])

    for idx, color_voxel in enumerate(color_volume):
        if (not np.array_equal(color_voxel,[0,0,0])):
            class_id, calss_name = rgb2nyudv2(color_voxel*255)
            classid_volume[idx] = class_id

    classid_volume = classid_volume.reshape([32,32,32])
    
    if (onehot):
        onehot_classid_volume = np.zeros([21,32,32,32])
        onehot_classid_volume = onehot_classid_volume.reshape([-1, 21])
        classid_volume = classid_volume.flatten()
        
        for onehot_classid_voxel, classid_voxel in zip(onehot_classid_volume, classid_volume):
            if (classid_voxel != 0): # for fast assignment
                onehot_classid_voxel[int(classid_voxel)] = 1
            else:
                onehot_classid_voxel[0] = 1
            
        onehot_classid_volume = onehot_classid_volume.reshape([21,32,32,32])
        classid_volume = onehot_classid_volume
    
    return classid_volume

def classidvol2colorvol(classid_volume):
    color_volume = np.zeros([32,32,32,3])
    color_volume = color_volume.reshape([-1,3])
    classid_volume = classid_volume.flatten()
    
    for idx, classid_voxel in enumerate(classid_volume):
        if (classid_voxel != 0):
            color_volume[idx] = list(nyudv2_color_codes.values())[int(classid_voxel)]
    
    color_volume = color_volume.reshape([32,32,32,3])
    
    return color_volume

In [4]:
def viz_vvae_output(output_color_volume, show=True, save=False):
    voxel_volume = np.zeros([32,32,32])
    voxel_volume = voxel_volume.flatten()
    color_volume = output_color_volume.reshape([-1,3])

    for idx, color_voxel in enumerate(color_volume):
        if (not np.array_equal(color_voxel, [0,0,0])):
            voxel_volume[idx] = 1

    voxel_volume = voxel_volume.reshape([32,32,32])
    color_volume = color_volume.reshape([32,32,32,3])
    color_volume = color_volume/256.
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')
    ax.voxels(voxel_volume, facecolors=color_volume)
    
    if (show): plt.show()
    if (save): 
        timestamp = str(datetime.datetime.utcnow())
        file_name = os.path.join('./result_image',timestamp+'.png')
        plt.savefig(file_name)
    
    plt.close()

# Model training

In [ ]:
vol_vae = VolumetricVAE(_kernel_size=3, _stride=1)
# vol_vae = torch.nn.DataParallel(vol_vae, device_ids = range(torch.cuda.device_count()))
# vol_vae.cuda()

optimizer = optim.SGD(vol_vae.parameters(), lr=0.0001, momentum=0.99, weight_decay=5e-4)
recon_loss_func = torch.nn.CrossEntropyLoss()

In [1]:
epoches = 100
data_range = 100

for epoch in range(epoches):
    for idx in range(data_range):
        incomp_pcd = read_point_cloud("/home/dongwonshin/Desktop/subvolume_dataset_making/mesh_results2/%06d_incomplete.ply" % idx)
        voxel_volume, color_volume = pcd2voxel(incomp_pcd)
        classid_volume = colorvol2classidvol(color_volume)
        classid_volume = np.expand_dims(classid_volume, axis=0)
        classid_volume = torch.from_numpy(classid_volume).float()

        optimizer.zero_grad()
        
        output, kl_loss = vol_vae.forward(classid_volume)

        comp_pcd = read_point_cloud("/home/dongwonshin/Desktop/subvolume_dataset_making/mesh_results2/%06d_complete.ply" % idx)
        _, target_volume = pcd2voxel(comp_pcd)
        target_volume = colorvol2classidvol(target_volume, False)
        target_volume = np.expand_dims(target_volume, axis=0)
        target_volume = torch.from_numpy(target_volume).long()
        
        recon_loss = recon_loss_func(output, target_volume)
        
        loss = recon_loss # + kl_loss
        
        loss.backward()
        optimizer.step()

        l = loss.data[0]

    # save the intermediate result
    argmaxed = torch.argmax(output,1)
    output_color_volume = classidvol2colorvol(argmaxed.numpy())
    viz_vvae_output(output_color_volume,show=False,save=True)
    
    # print loss
    print(l)

NameError: name 'read_point_cloud' is not defined